In [37]:
import pandas as pd
from time import time
from helper import *
from models import *
from cleaning import *
import fasttext.util

In [38]:
pos_train , neg_train = load_data()
#pos_train = pos_train.iloc[:2000]
#neg_train = neg_train.iloc[:2000]
X_train, X_test, y_train, y_test = split_data(pos_train, neg_train) #return pandas series

Train_set Info: SIZE= 157576, POSITIVE Tweets =49.71%, NEGATIVE Tweets = 50.29%
Test_set Info: SIZE= 39394, POSITIVE Tweets =49.69%, NEGATIVE Tweets = 50.31%


In [39]:
%%time
# CELL : avec SVM clean + Train = 15min pour 80.000 train set 
#clean  
X_train_clean= X_train.apply(clean_tweet)
X_test_clean = X_test.apply(clean_tweet)

CPU times: user 2min 5s, sys: 20.1 s, total: 2min 25s
Wall time: 2min 53s


In [40]:
X_train_clean

72642                         user exclamation sure good x
14644                                         user sup hun
83188    bestfriend love person make truly happy glad c...
69015                        user hey follow back question
60361    user dont like playing game unless ive played ...
                               ...                        
69964    laugh lost work exclamation fucking school com...
25324             luvlaughstein user stuff came mail today
65689                          user user user mag question
66804                               im reaching find youre
80450    someone named eva hardcover night nazi soldier...
Name: Text, Length: 157576, dtype: object

In [41]:
df_X_train_clean = X_train_clean.to_frame()
df_X_test_clean = X_test_clean.to_frame()
df_y_train = y_train.to_frame()
df_y_test = y_test.to_frame()

In [42]:
## get english word embeddings trained using fasttext
## size of word vectors in a hyperparameter cross validation to find best one
ft = fasttext.load_model('cc.en.300.bin')
ft.get_dimension()

300

In [71]:
#ft.get_nearest_neighbors('hello')
def get_word_emb(tweet):
    avg_vect = np.zeros(300)
    for word in tweet:
        avg_vect += ft.get_word_vector(word)
    return avg_vect / 300

tr_set = df_X_train_clean.apply(get_word_emb,axis=1).to_frame()
te_set = df_X_test_clean.apply(get_word_emb,axis=1).to_frame()
y_tr = df_y_train
y_te = df_y_test

In [36]:
y_tr = df_y_train
y_te = df_y_test
print(tr_set)

NameError: name 'tr_set' is not defined

In [72]:
word_vectors = tr_set[0].to_numpy()
corr_array = np.zeros(300)
corr_array = corr_array.reshape(300, -1)
for index in range(word_vectors.shape[0]):
   # print(word_vectors[index])
    corr_array = np.c_[corr_array, word_vectors[index]]
tr_set_correct = corr_array[:,1:].T
tr_set_correct.shape

(157576, 300)

In [73]:
def make_matrix(pandas_dataframe):
    word_vectors = pandas_dataframe[0].to_numpy()
    corr_array = np.zeros(300)
    corr_array = corr_array.reshape(300, -1)
    for index in range(word_vectors.shape[0]):
        corr_array = np.c_[corr_array, word_vectors[index]]
    return corr_array[:,1:].T

In [74]:
te_set_correct = make_matrix(te_set)
print(te_set_correct.shape)

(39394, 300)


In [43]:
# Reading the pickle object
file_to_read = open("tr_saved.pickle","rb")
tr_set_correct = pickle.load(file_to_read)
file_to_read.close()
file_to = open("te_saved.pickle","rb")
te_set_correct = pickle.load(file_to)
file_to.close()
print(tr_set_correct.shape)
print(te_set_correct.shape)

(157576, 300)
(39394, 300)


## Using SVM 

In [12]:
from sklearn import svm
classifier = svm.SVC()
classifier.fit(tr_set_correct, y_tr.to_numpy().reshape(-1))
#y_pred = clf.predict(te_set.array)

SVC()

In [ ]:
y_pred = classifier.predict(te_set_correct)

In [ ]:
(sum(y_pred == y_te.to_numpy().reshape(-1)) / y_pred.shape[0]) * 100

73.52896380159414

## Using Logistic regression

In [53]:
## make predictions in correct format for logistic regression 0,1 instead of -1, 1
y_tr_logi =  y_tr.to_numpy().reshape(-1)
y_te_logi = y_te.to_numpy().reshape(-1)
y_tr_logi[y_tr_logi == -1] = 0
y_te_logi[y_te_logi == -1] = 0

In [55]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression().fit(tr_set_correct,  y_tr_logi)
y_pred_log = classifier.predict(te_set_correct)
print(y_pred_log)
(sum(y_pred_log == y_te_logi) / y_pred_log.shape[0]) * 100

[0 0 0 ... 0 0 0]


50.31476874650962

In [48]:
y_pred_log[y_pred_log==0]

array([0, 0, 0, ..., 0, 0, 0])

In [84]:
# Reading the pickle object
#file_to_read_test = open("tr_saved.pickle","rb")
#pickle_test = pickle.load(file_to_read_test)
#file_to_read_test.close()
#print(pickle_test.shape)